In [4]:
import os 
import random
import glob
import tensorflow as tf
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D

from keras.callbacks import ModelCheckpoint
import numpy as np



In [6]:
def train_model (data_directory,checkpoint_path, batch_size, image_width, image_height, learning_rate):
    # Create data generators for training and validation
    datagen = ImageDataGenerator(rescale=1./255, rotation_range=90, validation_split= 0.1)

    train_generator = datagen.flow_from_directory(
        data_directory,
        target_size=(image_height, image_width),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical',
        subset='training')

    validation_generator = datagen.flow_from_directory(
        data_directory,
        target_size=( image_height, image_width),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation')

    num_classes = train_generator.num_classes
    
    # Define and compile your CNN model
    INPUT_SHAPE = ( image_width, image_height, 3)   #change to (SIZE, SIZE, 3)
    
    model = Sequential()

    model.add(Conv2D(32, (3, 3), 1, activation="relu", input_shape=(image_width, image_height, 3)))
    model.add(MaxPooling2D())
    model.add(Dropout(0.25))  # Add dropout with a rate of 25%

    model.add(Conv2D(48, (3, 3), 1, activation="relu"))
    model.add(MaxPooling2D())
    # model.add(Dropout(0.25))  # Add dropout with a rate of 25%

    model.add(Conv2D(32, (3, 3), 1, activation="relu"))
    model.add(MaxPooling2D())
    # model.add(Dropout(0.25))  # Add dropout with a rate of 25%

    model.add(Flatten())

    model.add(Dense(100, activation="relu"))

    # model.add(Dropout(0.5))  # Add dropout with a rate of 50%

    model.add(Dense(num_classes,activation="softmax"))  

    model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate),loss='categorical_crossentropy',metrics = ["accuracy"])
    print(model.summary())  
    
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = ModelCheckpoint(filepath=checkpoint_path, save_weights_only=False, verbose=1, save_best_only= True, monitor= "val_loss")

    # Train the model
    model.fit(train_generator, validation_data=validation_generator, epochs=5, callbacks=[cp_callback])



    # Evaluate the model on the validation set
    validation_images = []
    validation_labels = []
    for i in range(len(validation_generator)):
        batch_images, batch_labels = validation_generator[i]
        validation_images.append(batch_images)
        validation_labels.append(batch_labels)

    validation_images = tf.concat(validation_images, axis=0)
    validation_labels = tf.concat(validation_labels, axis=0)

    # Perform predictions on the validation set
    predictions = model.predict(validation_images)
    predicted_labels = tf.squeeze(tf.round(predictions))

    # Generate classification report

    classification_rep = classification_report(validation_labels, predicted_labels)
    print(classification_rep)
    
    







In [ ]:
data_directory = 'train'

checkpoint_path='C:\\Users\\Documents\\math_project\\model_archive'
#validation_data_directory = 'C:\\Users\\Documents\\validation_images'
#test_data_directory = 'C:\\Users\\Documents\\test_images'

# Define the batch size and image dimensions

batch_size = 5
image_width, image_height = 56 , 56
learning_rate=0.005
train_model(data_directory,checkpoint_path, batch_size, image_width, image_height, learning_rate)

Found 13169 images belonging to 3 classes.
Found 1461 images belonging to 3 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 54, 54, 32)        896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 27, 27, 32)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 27, 27, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 25, 25, 48)        13872     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 12, 12, 48)        0         
 g2D)                                                            
                                   

INFO:tensorflow:Assets written to: C:\Users\Documents\math_project\model_archive/assets


2634/2634 [==============================] - 101s 38ms/step - loss: 1.0992 - accuracy: 0.3444 - val_loss: 1.0997 - val_accuracy: 0.3238
Epoch 2/5
2633/2634 [============================>.] - ETA: 0s - loss: 1.0990 - accuracy: 0.3460
Epoch 2: val_loss improved from 1.09973 to 1.09804, saving model to C:\Users\Documents\math_project\model_archive
INFO:tensorflow:Assets written to: C:\Users\Documents\math_project\model_archive/assets


INFO:tensorflow:Assets written to: C:\Users\Documents\math_project\model_archive/assets


2634/2634 [==============================] - 99s 38ms/step - loss: 1.0989 - accuracy: 0.3460 - val_loss: 1.0980 - val_accuracy: 0.3525
Epoch 3/5
2633/2634 [============================>.] - ETA: 0s - loss: 1.0988 - accuracy: 0.3478
Epoch 3: val_loss did not improve from 1.09804
2634/2634 [==============================] - 102s 39ms/step - loss: 1.0988 - accuracy: 0.3479 - val_loss: 1.0983 - val_accuracy: 0.3525
Epoch 4/5
2634/2634 [==============================] - ETA: 0s - loss: 1.0987 - accuracy: 0.3476
Epoch 4: val_loss improved from 1.09804 to 1.09803, saving model to C:\Users\Documents\math_project\model_archive
INFO:tensorflow:Assets written to: C:\Users\Documents\math_project\model_archive/assets


INFO:tensorflow:Assets written to: C:\Users\Documents\math_project\model_archive/assets


2634/2634 [==============================] - 105s 40ms/step - loss: 1.0987 - accuracy: 0.3476 - val_loss: 1.0980 - val_accuracy: 0.3525
Epoch 5/5
2083/2634 [======================>.......] - ETA: 19s - loss: 1.0981 - accuracy: 0.3494